# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [16]:
import numpy as np
import pandas as pd 
from scipy import stats
df = pd.read_csv('homepage_actions.csv')
df['action'].value_counts()

view     6328
click    1860
Name: action, dtype: int64

In [75]:
df['count'] = 1

In [76]:
click_id = set(df[df['action'] == 'click']['id'].unique())
view_id = set(df[df['action'] == 'view']['id'].unique())
len(click_id), len(view_id)
print("Number of views:{}, Number of clicks:{}".format(len(view_id), len(click_id)))
print("Number of views without clicks:{}".format(len(view_id-click_id)))
print("Number of clicks without views:{}".format(len(click_id-view_id)))


Number of views:6328, Number of clicks:1860
Number of views without clicks:4468
Number of clicks without views:0


In [77]:
exp_id = set(df[df['group']=='experiment']['id'].unique())
con_id = set(df[df['group']=='control']['id'].unique())
print('Overlap of experiment and control groups: {}'.format(len(exp_id&con_id)))

Overlap of experiment and control groups: 0


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [93]:
cont = df[df['group']=='control']
exp = df[df['group']=='experiment']

cont = cont.pivot(index='id', columns='action', values='count')
exp = exp.pivot(index='id', columns= 'action', values ='count')
cont = cont.fillna(value=0)
exp = exp.fillna(value=0)

stat, p_value = stats.ttest_ind(cont['click'], exp['click'], equal_var=False)
p_value/2

0.004466402814337101

## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [119]:
cont_cr = cont['click'].mean()
expected_clicks = cont_cr*len(exp)
expected_clicks

838.0168067226891

### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [125]:
var = expected_clicks*(1-cont_cr)
std = np.sqrt(var)
act_clicks = sum(exp['click'])

answer = (act_clicks-expected_clicks)/std
answer





3.6625360854823588

### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [127]:
p_value= 1-stats.norm.cdf(answer)
p_value

0.00012486528006949715

### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **Your analysis here**
Roughly, yes. While there is a difference in the test results, both values are well within the range where we can
confidently reject the null hypothesis (the experimental page does not generate more clicks than the control page)

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.